In [2]:
import pandas as pd
import numpy as np

In [14]:
def split_data(path):
    save_path = "/".join(path.split('/')[:-1]) + '/'
    df = pd.read_csv(path)
    locations = df['location'].unique()
    print(path.split('/')[-1])
    for location in locations:
        tmp_df = pd.DataFrame()
        name = location + '_' + path.split('/')[-1]
        tmp_df = df[df['location'] == location]
        file_name = save_path + name
        #for test
        # file_name = save_path + '/test/' + name
        tmp_df = tmp_df.drop('location', axis = 1)
        tmp_df.to_csv(file_name) 

In [4]:
split_data('../data/pollen_train.csv')

pollen_train.csv


In [5]:
def merge_data(path, target = 'AMBROSIA', stride = 1):
    df = pd.read_csv(path)
    X = []
    y = []
    for i in range(0, df.shape[0] - 13, stride):
        lst = []

        ran = range(i, i + 10)
        X.append(np.array(df.iloc[ran]).flatten())
        y.append((
            float(df.iloc[[i+11]][target]),
            float(df.iloc[[i+12]][target]),
            float(df.iloc[[i+13]][target]),
        ))
    return np.array(X), y


In [6]:
df = pd.read_csv('../data/pollen_train.csv')
locations = df['location'].unique()

In [7]:
def XGB_input(path, locations, file, target = 'AMBROSIA', stride = 1):
    for location in locations:
        file_name = path + '/' + location + '_' + file
        save_path = path + '/XGB/' + location + '_' + file
        X, y = merge_data(file_name + '.csv',target = target, stride = stride)
        pd.DataFrame(X).to_csv(save_path + 'XGB.csv')
        pd.DataFrame(y).to_csv(save_path + 'XGB_target.csv')
    

In [8]:
XGB_input('../data', locations, 'pollen_train')

In [9]:
test1 = pd.read_csv('../data/pollen_train.csv')
weather_df = pd.read_csv('../data/weather_data_filtered.csv')
df = test1.merge(weather_df, how = 'left', on = (['location', 'date']))

In [10]:
df['prcp'] = df['prcp'].interpolate()
df['wspd'] = df['wspd'].interpolate(method = 'slinear')
df['wspd'] = df['wspd'].fillna(0)
df['wdir'] = df['wdir'].interpolate(method = 'slinear')
df['wdir'] = df['wdir'].fillna(0)
df = df.drop(['wpgt','snow','tsun','pres'], axis = 1)
df = df.dropna()


In [11]:
df.to_csv('../data/pollen_train_weather.csv')
split_data('../data/pollen_train_weather.csv')
XGB_input('../data', locations, 'pollen_train_weather')

pollen_train_weather.csv


In [13]:
df['location'].unique()

array(['БЕОГРАД - НОВИ БЕОГРАД', 'ВРШАЦ', 'НИШ', 'ПОЖАРЕВАЦ', 'СУБОТИЦА',
       'КРАГУЈЕВАЦ', 'КРАЉЕВО'], dtype=object)

In [15]:
split_data('../data/pollen_test.csv')

pollen_test.csv
